# Basics

There are two ways to create agents 🤖:

1. Using the Azure AI Agent Service SDK. **This is the recommended approach**.
2. Using the OpenAI SDK. In this case they are not called agents, but assistants.

The code below will import some Python classes and functions, will read the .env file, and get ready the environment.
In addition it will define the function pprint, which will be used in this workshop to improve the readability of the objects created.

In [18]:
import os
from pprint import pp as pp

from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Load environment variables from .env file
load_dotenv(override=True)

def pprint(obj):
    pp(obj.as_dict() if hasattr(obj, "as_dict") else obj, width=100)

# Print the environments we will be using.
print(f"PROJECT_CONNECTION_STRING: {os.getenv('PROJECT_CONNECTION_STRING')}")
print(f"BING_CONNECTION_NAME: {os.getenv('BING_CONNECTION_NAME')}")  

PROJECT_CONNECTION_STRING: 576a663a-07b7-4dd2-82d5-1347867b172a.workspace.eastus.api.azureml.ms;698d9beb-2466-404e-ae80-79d2faa013b1;nbm-eastus-rg;nbm-eastus-ai1
BING_CONNECTION_NAME: bing_grounding


Get the project reference using the current authenticated user and the connection string to the project where the agents will be created. 

In [17]:
project = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

## Creating Agents using Agent Service SDK

Now that we are connected to the project, let's create a very simple agent using **GPT-4o-mini** model as the agent _"brain"_. We are also providing instructions on how the agent should behave, adding a description and custom metadata fields. These metadata fields can be used to add custom information to find agents later for scenario with several agents.

In [29]:
portfolio_agent = project.agents.create_agent(
    model="gpt-4o-mini",
    name="Portfolio Guidance",
    instructions="You are a financial agent, providing portfolio guidance for users.",
    description="This agent was created to provide guidance for our users.",
    metadata= {
        "department": "finance",
        "owner": "jdoe"
    }
)
pprint(portfolio_agent)

{'id': 'asst_lHmmRLrLOJ051eikWpVYqVYD',
 'object': 'assistant',
 'created_at': 1739807752,
 'name': 'Portfolio Guidance',
 'description': 'This agent was created to provide guidance for our users.',
 'model': 'gpt-4o-mini',
 'instructions': 'You are a financial agent, providing portfolio guidance for users.',
 'tools': [],
 'top_p': 1.0,
 'temperature': 1.0,
 'tool_resources': {},
 'metadata': {'department': 'finance', 'owner': 'jdoe'},
 'response_format': 'auto'}


Every agent is unique and has an id, which starts with the string _asst_. This id and can be used to retrieve the agent later.

Let's create a second agent, but now for the marketing team.

In [30]:
branding_agent = project.agents.create_agent(
    model="gpt-4o-mini",
    name="Branding Guidance",
    instructions="You are a marketing agent, providing guidance about the correct way to use our branding. Make sure to warn users that our logo must be at the same size as the other company logos.",
    description="This agent was created to provide guidance for our users.",
    metadata={
        "department": "finance",
        "owner": "jdoe"
    }
)
pprint(branding_agent)

{'id': 'asst_lRObC2JvpO2eCHwX9DmbvWUK',
 'object': 'assistant',
 'created_at': 1739807756,
 'name': 'Branding Guidance',
 'description': 'This agent was created to provide guidance for our users.',
 'model': 'gpt-4o-mini',
 'instructions': 'You are a marketing agent, providing guidance about the correct way to use our '
                 'branding. Make sure to warn users that our logo must be at the same size as the '
                 'other company logos.',
 'tools': [],
 'top_p': 1.0,
 'temperature': 1.0,
 'tool_resources': {},
 'metadata': {'department': 'finance', 'owner': 'jdoe'},
 'response_format': 'auto'}


Using the project object it is possible to list all agents in the project, delete an agent or update an agent.

Right now we should have two agents created in the project.

In [31]:
pprint(project.agents.list_agents())

{'object': 'list',
 'data': [{'id': 'asst_lRObC2JvpO2eCHwX9DmbvWUK',
           'object': 'assistant',
           'created_at': 1739807756,
           'name': 'Branding Guidance',
           'description': 'This agent was created to provide guidance for our users.',
           'model': 'gpt-4o-mini',
           'instructions': 'You are a marketing agent, providing guidance about the correct way to '
                           'use our branding. Make sure to warn users that our logo must be at the '
                           'same size as the other company logos.',
           'tools': [],
           'top_p': 1.0,
           'temperature': 1.0,
           'tool_resources': {},
           'metadata': {'department': 'finance', 'owner': 'jdoe'},
           'response_format': 'auto'},
          {'id': 'asst_lHmmRLrLOJ051eikWpVYqVYD',
           'object': 'assistant',
           'created_at': 1739807752,
           'name': 'Portfolio Guidance',
           'description': 'This agent was creat

Let's improve the instructions we provided for the portfolio agent.

In [32]:
portfolio_agent = project.agents.update_agent(portfolio_agent.id, instructions="You are a financial agent, providing portfolio guidance for users. Avoid using too technical terms, and be clear about the risks.")
pprint(portfolio_agent)

{'id': 'asst_lHmmRLrLOJ051eikWpVYqVYD',
 'object': 'assistant',
 'created_at': 1739807752,
 'name': 'Portfolio Guidance',
 'description': 'This agent was created to provide guidance for our users.',
 'model': 'gpt-4o-mini',
 'instructions': 'You are a financial agent, providing portfolio guidance for users. Avoid using '
                 'too technical terms, and be clear about the risks.',
 'tools': [],
 'top_p': 1.0,
 'temperature': 1.0,
 'tool_resources': {},
 'metadata': {'department': 'finance', 'owner': 'jdoe'},
 'response_format': 'auto'}


### Chatting with the Agent

In order to chat with an agent, we need to create a **thread**. A thread is a **container** for messages, and there are two types of messages: messages created by the user or created by an agent.

**Important!**
1. If you don't save the thread id, you won't be able to recover the thread later. Keep this in mind, and save it to another database.
2. A thread is not tied to an agent. **The same thread can be used by several agents**.
3. Unless deleted manually, threads are kept forever.

Jane wants to get information on how to invest her money, so let's create a thread for Jane.
You can also add metadata for threads, and in this case we are adding the user id from Microsoft Entra.

In [33]:
thread_jane = project.agents.create_thread(
    metadata= {
        "entraUserId": "a1ddfd30-8420-4a8d-b155-4b5f05998d69"
    }
)
pprint(thread_jane)

{'id': 'thread_HPbFaNTL2BhOsh4i2Om5aNrx',
 'object': 'thread',
 'created_at': 1739807794,
 'metadata': {'entraUserId': 'a1ddfd30-8420-4a8d-b155-4b5f05998d69'},
 'tool_resources': {}}


Let's add a message to the thread we created.

In [34]:
message = project.agents.create_message(
    thread_id=thread_jane.id,
    role="user",
    content="What is the best option to invest my money?",
)
pprint(message)

{'id': 'msg_rWipqqjGKxy861tKkP38siZ6',
 'object': 'thread.message',
 'created_at': 1739807800,
 'assistant_id': None,
 'thread_id': 'thread_HPbFaNTL2BhOsh4i2Om5aNrx',
 'run_id': None,
 'role': 'user',
 'content': [{'type': 'text',
              'text': {'value': 'What is the best option to invest my money?', 'annotations': []}}],
 'attachments': [],
 'metadata': {}}


Let's print all messages in the thread.

In [35]:
pprint(project.agents.list_messages(thread_jane.id))

{'object': 'list',
 'data': [{'id': 'msg_rWipqqjGKxy861tKkP38siZ6',
           'object': 'thread.message',
           'created_at': 1739807800,
           'assistant_id': None,
           'thread_id': 'thread_HPbFaNTL2BhOsh4i2Om5aNrx',
           'run_id': None,
           'role': 'user',
           'content': [{'type': 'text',
                        'text': {'value': 'What is the best option to invest my money?',
                                 'annotations': []}}],
           'attachments': [],
           'metadata': {}}],
 'first_id': 'msg_rWipqqjGKxy861tKkP38siZ6',
 'last_id': 'msg_rWipqqjGKxy861tKkP38siZ6',
 'has_more': False}


At this point, we created a thread containing only one message. As you can see, the thread or message are not linked to any assistant. You can validate that looking the **run_id** and **assistant_id** properties for messages where the role is **user**.

Now, it's time to ask the agent to reason over the message the user sent and generate a reply. This is represented by a run object. 

The execution of the agent is **asynchronous** and we need to poll if its execution was completed. The function **create_and_process_run** abstracts all this process.

In [36]:
pprint(project.agents.create_and_process_run(thread_id=thread_jane.id, assistant_id=portfolio_agent.id)) 

{'id': 'run_hRxlTlSlYA8XGvyhgzK7pMXo',
 'object': 'thread.run',
 'created_at': 1739807807,
 'assistant_id': 'asst_lHmmRLrLOJ051eikWpVYqVYD',
 'thread_id': 'thread_HPbFaNTL2BhOsh4i2Om5aNrx',
 'status': 'completed',
 'started_at': 1739807807,
 'expires_at': None,
 'cancelled_at': None,
 'failed_at': None,
 'completed_at': 1739807811,
 'required_action': None,
 'last_error': None,
 'model': 'gpt-4o-mini',
 'instructions': 'You are a financial agent, providing portfolio guidance for users. Avoid using '
                 'too technical terms, and be clear about the risks.',
 'tools': [],
 'tool_resources': {},
 'metadata': {},
 'temperature': 1.0,
 'top_p': 1.0,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'incomplete_details': None,
 'usage': {'prompt_tokens': 81,
           'completion_tokens': 487,
           'total_tokens': 568,
           'prompt_token_details': {'cached_tokens': 0}},
 'response_format': '

Now, let's list the message that was created and added to the thread. You will see that the agent added a new message to the list.

In [37]:
pprint(project.agents.list_messages(thread_jane.id))

{'object': 'list',
 'data': [{'id': 'msg_mTyrPnJsIKYh8WHl9DcvIjw4',
           'object': 'thread.message',
           'created_at': 1739807808,
           'assistant_id': 'asst_lHmmRLrLOJ051eikWpVYqVYD',
           'thread_id': 'thread_HPbFaNTL2BhOsh4i2Om5aNrx',
           'run_id': 'run_hRxlTlSlYA8XGvyhgzK7pMXo',
           'role': 'assistant',
           'content': [{'type': 'text',
                        'text': {'value': 'The best option for investing your money depends on '
                                          'your personal financial goals, risk tolerance, time '
                                          'horizon, and current financial situation. Here are some '
                                          'common investment avenues, along with their potential '
                                          'risks and benefits:\n'
                                          '\n'
                                          '1. **Savings Account**: \n'
                                  

## Creating Agents using OpenAI SDK

Another way of creating agents is using OpenAI SDK. In this case, they are called **assistants**. 

To create one, we need an Azure OpenAI deployment reference, which can be acquired using the following code.

In [38]:
openai_client = project.inference.get_azure_openai_client(
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)

Now, let's create an assistant to help users about branding questions using this client.

In [39]:
branding_agent = openai_client.beta.assistants.create(
    model="gpt-4o-mini",
    name="Branding Guidance",
    instructions="You are a marketing agent, providing guidance about the correct way to use our branding. Make sure to warn users that our logo must be at the same size as the other company logos."
)
pprint(branding_agent)

Assistant(id='asst_9GQFmRbTcVYz7L3pjdixOiav', created_at=1739807829, description=None, instructions='You are a marketing agent, providing guidance about the correct way to use our branding. Make sure to warn users that our logo must be at the same size as the other company logos.', metadata={}, model='gpt-4o-mini', name='Branding Guidance', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)


The same way we did using Agent Service SDK, we can use the OpenAI SDK to list, update or delete assistants.

In [40]:
pprint(openai_client.beta.assistants.list())

SyncCursorPage[Assistant](data=[Assistant(id='asst_9GQFmRbTcVYz7L3pjdixOiav', created_at=1739807829, description=None, instructions='You are a marketing agent, providing guidance about the correct way to use our branding. Make sure to warn users that our logo must be at the same size as the other company logos.', metadata={}, model='gpt-4o-mini', name='Branding Guidance', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)], has_more=False, object='list', first_id='asst_9GQFmRbTcVYz7L3pjdixOiav', last_id='asst_9GQFmRbTcVYz7L3pjdixOiav')


Maria has some questions about how the company logo can be used. We will create using a single call a thread with a list of messages. Previously we created this in two differente calls.

In [41]:
thread_maria = openai_client.beta.threads.create(
    messages= [
        {
            "role": "user",
            "content": "I need to add the company logo side by side with another company logo. Is there any guidance about it?",         
        }
    ]
)
pprint(thread_maria)

Thread(id='thread_pdX38TmIjO0QJvI8hfsPfluh', created_at=1739807834, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


Execute the assistant over the thread.

In [42]:
run = openai_client.beta.threads.runs.create(
    thread_id=thread_maria.id,
    assistant_id=branding_agent.id,
)
pprint(run)

Run(id='run_FAVnjQxPrDA9VRbNc6zJQHGV', assistant_id='asst_9GQFmRbTcVYz7L3pjdixOiav', cancelled_at=None, completed_at=None, created_at=1739807836, expires_at=1739808436, failed_at=None, incomplete_details=None, instructions='You are a marketing agent, providing guidance about the correct way to use our branding. Make sure to warn users that our logo must be at the same size as the other company logos.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_pdX38TmIjO0QJvI8hfsPfluh', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


Print the message created by the assistant.

In [43]:
pprint(openai_client.beta.threads.messages.list(thread_id=thread_maria.id))

SyncCursorPage[Message](data=[Message(id='msg_z3FER0QIZXU56NV6mSWc1RcU', assistant_id='asst_9GQFmRbTcVYz7L3pjdixOiav', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Absolutely! When placing our logo alongside another company logo, please adhere to the following guidelines to ensure proper representation of our brand:\n\n1. **Equal Sizing:** Our logo must be the same size as the other company logos to maintain visual balance. Avoid enlarging or reducing our logo in a way that makes it appear dominant or less significant compared to the others.\n\n2. **Clear Space:** Ensure there is sufficient clear space around our logo, free from any other text or images. This helps enhance visibility and preserves brand integrity.\n\n3. **Placement:** Position the logos on a neutral background that does not clash with either brand, ensuring both logos can be clearly distinguished.\n\n4. **Alignment:** Align the logos so they are visually harmonious, whet

As you can see using Agent Service SDK or OpenAI SDK the experience today is almost the same, however the recomendation is to use the Agent Service SDK.


You have reached the end of this lab. 👏